In [10]:

'''Trains a Siamese MLP on pairs of digits from the MNIST dataset.
It follows Hadsell-et-al.'06 [1] by computing the Euclidean distance on the
output of the shared network and by optimizing the contrastive loss (see paper
for more details).

# References
- Dimensionality Reduction by Learning an Invariant Mapping
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
Gets to 97.2% test accuracy after 20 epochs.
2 seconds per epoch on a Titan X Maxwell GPU
'''
from __future__ import absolute_import
from __future__ import print_function
import numpy as np

import random
import keras
from keras.datasets import mnist
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout, Lambda
from keras.optimizers import RMSprop
from keras import backend as K
import tensorflow as tf

num_classes = 10
epochs = 20
print(keras.__version__)
print(K.tf.__version__)
#print(dir(K))
#print(dir(K.tf))

2.2.3
1.10.0


In [33]:
def euclidean_distance(vects):
    x, y = vects
    sum_square = K.sum(K.square(x - y), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sum_square, K.epsilon()))


def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)


def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    square_pred = K.square(y_pred)
    margin_square = K.square(K.maximum(margin - y_pred, 0))
    return K.mean(y_true * square_pred + (1 - y_true) * margin_square)

def double_contrastive_loss(y,d):
    '''Double contrastive loss from me 2019
    
    '''
    # splitting data
    y_e,y_r= K.tf.split(y,2, axis=-1)
    d_pred_e,d_pred_r = K.tf.split(d,2, axis=-1)

    # margins for relative is little since it is in the ball of normal margin
    margin_e = 1.0
    margin_r = 0.2
    
    # same importance const
    cs=margin_e/margin_r
    
    square_pred_e = K.square(d_pred_e)
    square_pred_r = K.square(d_pred_r)
    
    margin_square_e = K.square(K.maximum(margin_e - d_pred_e, 0))
    margin_square_r = K.square(K.maximum(margin_r - d_pred_r, 0))
    
    # relative loss from embedding vector differences
    loss_r=y_r * square_pred_r + (1 - y_r) * margin_square_r
    # main los from embedding vectors
    loss_e=y_e * square_pred_e + (1 - y_e) * margin_square_e 
    # merging losses
    loss=K.mean(loss_e + cs * loss_r)
    return loss


In [13]:
def create_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)

#TODO
def create_double_pairs(x, digit_indices):
    '''Positive and negative pair creation.
    Alternates between positive and negative pairs.
    '''
    pairs = []
    labels = []
    n = min([len(digit_indices[d]) for d in range(num_classes)]) - 1
    for d in range(num_classes):
        for i in range(n):
            z1, z2 = digit_indices[d][i], digit_indices[d][i + 1]
            pairs += [[x[z1], x[z2]]]
            inc = random.randrange(1, num_classes)
            dn = (d + inc) % num_classes
            z1, z2 = digit_indices[d][i], digit_indices[dn][i]
            pairs += [[x[z1], x[z2]]]
            labels += [1, 0]
    return np.array(pairs), np.array(labels)


In [14]:
def create_base_network(input_shape):
    '''Base network to be shared (eq. to feature extraction).
    '''
    input = Input(shape=input_shape)
    x = Flatten()(input)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.1)(x)
    x = Dense(128, activation='relu')(x)
    #in order to visualize reduce dimension to 2
    x = Dense(  2, activation='relu')(x)
    return Model(input, x)


In [15]:
def compute_accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    pred = y_pred.ravel() < 0.5
    return np.mean(pred == y_true)

#how work out accuracy for double contactive loss?
def accuracy(y_true, y_pred):
    '''Compute classification accuracy with a fixed threshold on distances.
    '''
    return K.mean(K.equal(y_true, K.cast(y_pred < 0.5, y_true.dtype)))


In [16]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
input_shape = x_train.shape[1:]


In [17]:
# create training+test positive and negative pairs
digit_indices = [np.where(y_train == i)[0] for i in range(num_classes)]
tr_pairs, tr_y = create_pairs(x_train, digit_indices)

digit_indices = [np.where(y_test == i)[0] for i in range(num_classes)]
te_pairs, te_y = create_pairs(x_test, digit_indices)


In [18]:
# network definition
base_network = create_base_network(input_shape)

input_a0 = Input(shape=input_shape)
input_b0 = Input(shape=input_shape)
input_a1 = Input(shape=input_shape)
input_b1 = Input(shape=input_shape)

# because we re-use the same instance `base_network`,
# the weights of the network
# will be shared across the two branches
processed_a0 = base_network(input_a0)
processed_b0 = base_network(input_b0)
processed_a1 = base_network(input_a1)
processed_b1 = base_network(input_b1)

distance_e = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([processed_a0, processed_b0])

da = keras.layers.Subtract()([processed_a1,processed_a0])
db = keras.layers.Subtract()([processed_b1,processed_b0])

distance_r = Lambda(euclidean_distance,
                  output_shape=eucl_dist_output_shape)([da,db])

o=keras.layers.Concatenate()([distance_e,distance_r])

model = Model([input_a0, input_b0, input_a1, input_b1], o)

encoder=base_network(Input(shape=input_shape))

In [34]:
# train
rms = RMSprop()
model.compile(loss=double_contrastive_loss, optimizer=rms, metrics=[accuracy])

In [10]:
model.fit([tr_pairs[:, 0], tr_pairs[:, 1],tr_pairs[:, 2], tr_pairs[:, 3]], [tr_y[0],tr_y[1]],
          batch_size=128,
          nb_epoch=epochs,
          validation_data=([te_pairs[:, 0], te_pairs[:, 1]], te_y))

TypeError: Tensor objects are not iterable when eager execution is not enabled. To iterate over this tensor use tf.map_fn.

In [12]:
# compute final accuracy on training and test sets
y_pred = model.predict([tr_pairs[:, 0], tr_pairs[:, 1]])
tr_acc = compute_accuracy(tr_y, y_pred)
y_pred = model.predict([te_pairs[:, 0], te_pairs[:, 1]])
te_acc = compute_accuracy(te_y, y_pred)

print('* Accuracy on training set: %0.2f%%' % (100 * tr_acc))
print('* Accuracy on test set: %0.2f%%' % (100 * te_acc))

* Accuracy on training set: 99.60%
* Accuracy on test set: 97.19%


In [ ]:
#TOFINISH
# display a 2D plot of the digit classes in the latent space
x_test_encoded = encoder.predict(x_test, batch_size=batch_size)
plt.figure(figsize=(6, 6))
plt.scatter(x_test_encoded[:, 0], x_test_encoded[:, 1], c=y_test)
plt.colorbar()
plt.show()